In [1]:
!pip install langchain==0.0.330
!pip install openai==0.28.1
!pip install PyPDF2
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.4 MB/s eta 0:00:00


In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [16]:
import os
os.environ['OPENAI_API_KEY'] = '<insert_api_key>'

In [17]:
reader = PdfReader("/content/Genome.pdf")

In [18]:
reader

In [19]:
raw_text = ''
for i, page in enumerate(reader.pages):
  text = page.extract_text()
  if text:
    raw_text += text

In [20]:
len(raw_text)

43241

In [21]:
raw_text[0:100]

'1 Scientific  RepoRtS  |         (2019) 9:17256  | https://doi.org/10.1038/s41598-019-53989-3\nwww.na'

In [22]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [23]:
embeddings = OpenAIEmbeddings()

In [24]:
!pip install tiktoken --quiet

In [25]:
docsearch = FAISS.from_texts(texts, embeddings)

In [52]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: You are a bot who is assisting a university for any queries that a student may have. If the input message asks you to generate a question for
a specified number of marks, please generate the question in accordance to some examples given below as the questions are framed with the number of marks
in mind.Do not use technical words, give easy to understand responses
Question: {query}


Answer: """
prompt_template = PromptTemplate(
    input_variables = ['query'],
    template = template
)

In [53]:
def combined_prompt(imp):
  return template.format(query = imp)

In [54]:
llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature = 1.0)
chain = load_qa_chain(llm, chain_type = "stuff")

In [55]:
from IPython.display import Markdown
query = "give a 2 mark answer on types of cancer"
query = combined_prompt(query)
docs = docsearch.similarity_search(query)
y = chain.run(input_documents = docs, question = query)
Markdown(y)

There are 12 types of cancer mentioned in the provided context, including breast cancer, colon cancer, glioblastoma multiforme, lung cancer, ovarian cancer, prostate cancer, skin melanoma, thyroid cancer, uterine cancer, and others.